In [32]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from scipy import stats
from scipy.stats import randint 
from scipy.stats import uniform
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

In [33]:
dataset = pd.read_excel('Eye_measurements_may_2020.xls')
dataset.shape

(57271, 18)

In [34]:
dataset.head(10)

,Externe ID,Geslacht,Geboortedatum,Oogmetingen/Datum,Oogmetingen/Sfr ver,Oogmetingen/Cyl ver,Oogmetingen/As ver,Oogmetingen/Add,Oogmetingen/Sfr dicht,Oogmetingen/Cyl dicht,Oogmetingen/As dicht,Oogmetingen/Sfr ver.1,Oogmetingen/Cyl ver.1,Oogmetingen/As ver.1,Oogmetingen/Add.1,Oogmetingen/Sfr dicht.1,Oogmetingen/Cyl dicht.1,Oogmetingen/As dicht.1
0,__export__.res_partner_10972_f0a5feb4,Vrouw,1962-07-24 00:00:00,2020-05-18 00:00:00,-1.25,-0.50,145.0,NaN,NaN,-1.0,145.0,-0.75,-1.00,20.0,NaN,NaN,-0.50,20.0
1,NaN,NaN,NaN,2017-12-27 00:00:00,-1.25,-0.50,155.0,0.0,0.00,0.0,0.0,-0.75,-0.50,20.0,0.0,0.00,0.00,0.0
2,NaN,NaN,NaN,2012-02-11 00:00:00,-1.00,-0.50,155.0,0.0,0.00,0.0,0.0,-0.50,-0.50,20.0,0.0,0.00,0.00,0.0
3,__export__.res_partner_7722_2342b746,Man,1982-07-10 00:00:00,2020-05-16 00:00:00,-2.00,-1.50,90.0,NaN,NaN,NaN,NaN,-1.50,-1.50,90.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2008-12-13 00:00:00,-1.25,-0.75,85.0,0.0,0.00,0.0,0.0,-1.25,-0.75,90.0,0.0,0.00,0.00,0.0
5,NaN,NaN,NaN,2001-04-06 00:00:00,-1.00,-0.50,80.0,0.0,0.00,0.0,0.0,-1.00,-0.50,80.0,0.0,0.00,0.00,0.0
6,NaN,NaN,NaN,1998-05-06 00:00:00,-0.25,-0.50,80.0,0.0,0.00,0.0,0.0,-0.50,-0.50,80.0,0.0,0.00,0.00,0.0
7,__export__.res_partner_12291_d15d7cb3,Vrouw,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,__export__.res_partner_10292_fd17c1d1,False,NaN,2019-08-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,__export__.res_partner_4452_c8f47e75,Vrouw,1950-03-18 00:00:00,2020-05-16 00:00:00,2.25,0.75,90.0,3.0,5.25,1.0,90.0,2.75,1.00,95.0,3.0,5.75,0.75,95.0


In [35]:
#drop alle rijen waar enkel Nan waarden zitten (in subset)
dataset = dataset.dropna(thresh=1,subset=['Oogmetingen/Sfr ver','Oogmetingen/Cyl ver',
                                            'Oogmetingen/As ver','Oogmetingen/Add',
                                            'Oogmetingen/Sfr dicht','Oogmetingen/Cyl dicht',
                                            'Oogmetingen/As dicht','Oogmetingen/Sfr ver.1',
                                            'Oogmetingen/Cyl ver.1','Oogmetingen/As ver.1',
                                            'Oogmetingen/Add.1','Oogmetingen/Sfr dicht.1',
                                            'Oogmetingen/Cyl dicht.1','Oogmetingen/As dicht.1'])

In [36]:
#controleren of een add.1 beschikbaar is en een add niet of omgekeerd en die dan aanvullen, als beide niet beschikbaar zijn 0 invoegen
values = {'Oogmetingen/Add': dataset['Oogmetingen/Add.1']}
dataset = dataset.fillna(value=values)
values = {'Oogmetingen/Add.1': dataset['Oogmetingen/Add']}
dataset = dataset.fillna(value=values)
values = {'Oogmetingen/Add.1': 0,
          'Oogmetingen/Add':0}
dataset = dataset.fillna(value=values)

In [37]:
#sfr dicht = sfr ver + add
#sfr ver = sfr dicht - add
values = {'Oogmetingen/Sfr dicht': dataset['Oogmetingen/Sfr ver'].add(dataset['Oogmetingen/Add']),
          'Oogmetingen/Sfr dicht.1':dataset['Oogmetingen/Sfr ver.1'].add(dataset['Oogmetingen/Add.1']) }
dataset = dataset.fillna(value=values)
values = {'Oogmetingen/Sfr ver': dataset['Oogmetingen/Sfr dicht'].add(dataset['Oogmetingen/Add']),
          'Oogmetingen/Sfr ver.1':dataset['Oogmetingen/Sfr dicht.1'].add(dataset['Oogmetingen/Add.1']) }
dataset = dataset.fillna(value=values)

In [38]:
#cyl en as dicht  = cyl en as ver en vice versa
values = {'Oogmetingen/Cyl dicht': dataset['Oogmetingen/Cyl ver'],
          'Oogmetingen/As dicht':dataset['Oogmetingen/As ver'],
          'Oogmetingen/Cyl dicht.1': dataset['Oogmetingen/Cyl ver.1'],
          'Oogmetingen/As dicht.1':dataset['Oogmetingen/As ver.1'] }
dataset = dataset.fillna(value=values)
values = {'Oogmetingen/Cyl ver': dataset['Oogmetingen/Cyl dicht'],
          'Oogmetingen/As ver':dataset['Oogmetingen/As dicht'],
          'Oogmetingen/Cyl ver.1': dataset['Oogmetingen/Cyl dicht.1'],
          'Oogmetingen/As ver.1':dataset['Oogmetingen/As dicht.1'] }
dataset = dataset.fillna(value=values)

In [39]:
#fill overige NaN waardes met 0
values = {'Oogmetingen/Sfr ver': 0, 'Oogmetingen/Cyl ver': 0, 'Oogmetingen/As ver':0 ,'Oogmetingen/Add':0,'Oogmetingen/Sfr dicht':0,'Oogmetingen/Cyl dicht':0,'Oogmetingen/As dicht':0,'Oogmetingen/Sfr ver.1':0,'Oogmetingen/Cyl ver.1':0,'Oogmetingen/As ver.1':0,'Oogmetingen/Add.1':0,'Oogmetingen/Sfr dicht.1':0,'Oogmetingen/Cyl dicht.1':0,'Oogmetingen/As dicht.1':0}
dataset = dataset.fillna(value=values)

In [40]:

dictSfr = {}  
dictCyl = {}
dictAs = {}
dictSfrDicht = {}  
dictCylDicht = {}
dictAsDicht = {}
dictSfr1 = {}  
dictCyl1 = {}
dictAs1 = {}
dictSfr1Dicht = {}  
dictCyl1Dicht = {}
dictAs1Dicht = {}

for index, row in dataset.iterrows():
    if row['Oogmetingen/Cyl ver']<0:
        sfr = row['Oogmetingen/Sfr ver']+row['Oogmetingen/Cyl ver']
        cyl = row['Oogmetingen/Cyl ver']-2*row['Oogmetingen/Cyl ver']
        key = index
        dictSfr[key] = sfr
        dictCyl[key] = cyl
        if row['Oogmetingen/As ver'] <= 90:
            dictAs[key] = row['Oogmetingen/As ver']+90
        elif row['Oogmetingen/As ver'] > 90:
            dictAs[key] = row['Oogmetingen/As ver']-90
    if row['Oogmetingen/Cyl dicht']<0:
        sfrDicht = row['Oogmetingen/Sfr dicht']+row['Oogmetingen/Cyl dicht']
        cylDicht = row['Oogmetingen/Cyl dicht']-2*row['Oogmetingen/Cyl dicht']
        key = index
        dictSfrDicht[key] = sfrDicht
        dictCylDicht[key] = cylDicht
        if row['Oogmetingen/As dicht'] <= 90:
            dictAsDicht[key] = row['Oogmetingen/As dicht']+90
        elif row['Oogmetingen/As dicht'] > 90:
            dictAsDicht[key] = row['Oogmetingen/As dicht']-90
    if row['Oogmetingen/Cyl ver.1']<0:
        sfr1 = row['Oogmetingen/Sfr ver.1']+row['Oogmetingen/Cyl ver.1']
        cyl1 = row['Oogmetingen/Cyl ver.1']-2*row['Oogmetingen/Cyl ver.1']
        key = index
        dictSfr1[key] = sfr1
        dictCyl1[key] = cyl1
        if row['Oogmetingen/As ver.1'] <= 90:
            dictAs1[key] = row['Oogmetingen/As ver.1']+90
        elif row['Oogmetingen/As ver.1'] > 90:
            dictAs1[key] = row['Oogmetingen/As ver.1']-90
    if row['Oogmetingen/Cyl dicht.1']<0:
        sfr1Dicht = row['Oogmetingen/Sfr dicht.1']+row['Oogmetingen/Cyl dicht.1']
        cyl1Dicht = row['Oogmetingen/Cyl dicht.1']-2*row['Oogmetingen/Cyl dicht.1']
        key = index
        dictSfr1Dicht[key] = sfr1Dicht
        dictCyl1Dicht[key] = cyl1Dicht
        if row['Oogmetingen/As dicht.1'] <= 90:
            dictAs1Dicht[key] = row['Oogmetingen/As dicht.1']+90
        elif row['Oogmetingen/As dicht.1'] > 90:
            dictAs1Dicht[key] = row['Oogmetingen/As dicht.1']-90

dataset['Oogmetingen/Sfr ver'].update(pd.Series(dictSfr))
dataset['Oogmetingen/Cyl ver'].update(pd.Series(dictCyl))
dataset['Oogmetingen/As ver'].update(pd.Series(dictAs))
dataset['Oogmetingen/Sfr dicht'].update(pd.Series(dictSfrDicht))
dataset['Oogmetingen/Cyl dicht'].update(pd.Series(dictCylDicht))
dataset['Oogmetingen/As dicht'].update(pd.Series(dictAsDicht))
dataset['Oogmetingen/Sfr ver.1'].update(pd.Series(dictSfr1))
dataset['Oogmetingen/Cyl ver.1'].update(pd.Series(dictCyl1))
dataset['Oogmetingen/As ver.1'].update(pd.Series(dictAs1))
dataset['Oogmetingen/Sfr dicht.1'].update(pd.Series(dictSfr1Dicht))
dataset['Oogmetingen/Cyl dicht.1'].update(pd.Series(dictCyl1Dicht))
dataset['Oogmetingen/As dicht.1'].update(pd.Series(dictAs1Dicht))

In [41]:
dataset.head(10)

,Externe ID,Geslacht,Geboortedatum,Oogmetingen/Datum,Oogmetingen/Sfr ver,Oogmetingen/Cyl ver,Oogmetingen/As ver,Oogmetingen/Add,Oogmetingen/Sfr dicht,Oogmetingen/Cyl dicht,Oogmetingen/As dicht,Oogmetingen/Sfr ver.1,Oogmetingen/Cyl ver.1,Oogmetingen/As ver.1,Oogmetingen/Add.1,Oogmetingen/Sfr dicht.1,Oogmetingen/Cyl dicht.1,Oogmetingen/As dicht.1
0,__export__.res_partner_10972_f0a5feb4,Vrouw,1962-07-24 00:00:00,2020-05-18 00:00:00,-1.75,0.50,55.0,0.0,-2.25,1.0,55.0,-1.75,1.00,110.0,0.0,-1.25,0.50,110.0
1,NaN,NaN,NaN,2017-12-27 00:00:00,-1.75,0.50,65.0,0.0,0.00,0.0,0.0,-1.25,0.50,110.0,0.0,0.00,0.00,0.0
2,NaN,NaN,NaN,2012-02-11 00:00:00,-1.50,0.50,65.0,0.0,0.00,0.0,0.0,-1.00,0.50,110.0,0.0,0.00,0.00,0.0
3,__export__.res_partner_7722_2342b746,Man,1982-07-10 00:00:00,2020-05-16 00:00:00,-3.50,1.50,180.0,0.0,-3.50,1.5,180.0,-3.00,1.50,180.0,0.0,-3.00,1.50,180.0
4,NaN,NaN,NaN,2008-12-13 00:00:00,-2.00,0.75,175.0,0.0,0.00,0.0,0.0,-2.00,0.75,180.0,0.0,0.00,0.00,0.0
5,NaN,NaN,NaN,2001-04-06 00:00:00,-1.50,0.50,170.0,0.0,0.00,0.0,0.0,-1.50,0.50,170.0,0.0,0.00,0.00,0.0
6,NaN,NaN,NaN,1998-05-06 00:00:00,-0.75,0.50,170.0,0.0,0.00,0.0,0.0,-1.00,0.50,170.0,0.0,0.00,0.00,0.0
9,__export__.res_partner_4452_c8f47e75,Vrouw,1950-03-18 00:00:00,2020-05-16 00:00:00,2.25,0.75,90.0,3.0,5.25,1.0,90.0,2.75,1.00,95.0,3.0,5.75,0.75,95.0
10,NaN,NaN,NaN,2011-04-28 00:00:00,2.75,0.75,90.0,2.5,5.25,2.0,90.0,2.75,2.00,95.0,2.5,5.25,0.75,95.0
11,NaN,NaN,NaN,2010-06-28 00:00:00,2.75,0.75,90.0,2.5,5.25,2.0,90.0,2.75,2.00,95.0,2.5,5.25,0.75,95.0


In [42]:
dataset.describe()

,Oogmetingen/Sfr ver,Oogmetingen/Cyl ver,Oogmetingen/As ver,Oogmetingen/Add,Oogmetingen/Sfr dicht,Oogmetingen/Cyl dicht,Oogmetingen/As dicht,Oogmetingen/Sfr ver.1,Oogmetingen/Cyl ver.1,Oogmetingen/As ver.1,Oogmetingen/Add.1,Oogmetingen/Sfr dicht.1,Oogmetingen/Cyl dicht.1,Oogmetingen/As dicht.1
count,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000,55454.000000
mean,2.363268,0.455423,50.042612,0.638711,2.039395,0.309978,37.981733,1.734857,0.459933,48.328020,0.639081,1.714217,0.309344,36.602301
std,52.177619,0.727860,64.118522,1.063664,33.739713,0.609959,65.230726,46.154544,0.736074,63.517943,1.064081,28.859194,0.717607,58.853923
min,-26.000000,0.000000,0.000000,-0.750000,-25.000000,0.000000,0.000000,-27.750000,0.000000,0.000000,-0.750000,-23.500000,0.000000,0.000000
25%,-1.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.250000,0.750000,90.000000,1.500000,2.500000,0.500000,90.000000,1.250000,0.750000,90.000000,1.500000,2.500000,0.500000,90.000000
max,1001.490000,7.000000,180.000000,8.500000,1002.990000,20.000000,6280.000000,999.990000,20.000000,180.000000,8.500000,1000.990000,90.000000,180.000000
